In [1]:
import os
import requests
from dotenv import load_dotenv
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool

c:\CS\WEATHER_FORECAST_AGENT_21_04\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Load environment variables
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OWM_API_KEY = os.getenv("OWM_API_KEY")

In [3]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    temperature=0.0,
    api_key=GROQ_API_KEY
)

In [4]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

In [5]:
city = "Pune"
units = "metric"

params = {
    "q": city,
    "appid": OWM_API_KEY,
    "units": units
}

In [6]:
response = requests.get(BASE_URL, params=params)

In [7]:
print(response.json())

{'coord': {'lon': 73.8553, 'lat': 18.5196}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 38.41, 'feels_like': 36.17, 'temp_min': 38.41, 'temp_max': 38.41, 'pressure': 1008, 'humidity': 15, 'sea_level': 1008, 'grnd_level': 936}, 'visibility': 10000, 'wind': {'speed': 2.93, 'deg': 21, 'gust': 2.06}, 'clouds': {'all': 0}, 'dt': 1745225250, 'sys': {'country': 'IN', 'sunrise': 1745196204, 'sunset': 1745241769}, 'timezone': 19800, 'id': 1259229, 'name': 'Pune', 'cod': 200}


In [8]:
## Weather forecast function

def get_weather_forecast(city: str, units: str = "metric") -> dict:
    """This function returns the current weather forecast for the given city."""
    params = {
        "q": city,
        "appid": OWM_API_KEY,
        "units": units
    }
    
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    
    return result


In [9]:
output = get_weather_forecast("Pune")
print(output)

{'coord': {'lon': 73.8553, 'lat': 18.5196}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 38.41, 'feels_like': 36.17, 'temp_min': 38.41, 'temp_max': 38.41, 'pressure': 1008, 'humidity': 15, 'sea_level': 1008, 'grnd_level': 936}, 'visibility': 10000, 'wind': {'speed': 2.93, 'deg': 21, 'gust': 2.06}, 'clouds': {'all': 0}, 'dt': 1745225250, 'sys': {'country': 'IN', 'sunrise': 1745196204, 'sunset': 1745241769}, 'timezone': 19800, 'id': 1259229, 'name': 'Pune', 'cod': 200}


In [10]:
## Converting the function to a function tool

weather_forecast_tool = FunctionTool.from_defaults(
    fn=get_weather_forecast,
    name="get_weather_forecast",
    description="This function returns the current weather forecast for the given city."
)

In [11]:
tools_list = [weather_forecast_tool] ## Can add additional functions to the list

In [12]:
agent = ReActAgent.from_tools(
    tools_list,
    llm=llm,
    verbose=True
)

In [13]:
response = agent.chat("What is the temperature in Pune?")
print(response)

> Running step 2d1fc9c6-1601-4b1b-b828-f4ba60a08981. Step input: What is the temperature in Pune?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather_forecast
Action Input: {'city': 'Pune', 'units': 'metric'}
Observation: {'coord': {'lon': 73.8553, 'lat': 18.5196}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 38.41, 'feels_like': 36.17, 'temp_min': 38.41, 'temp_max': 38.41, 'pressure': 1008, 'humidity': 15, 'sea_level': 1008, 'grnd_level': 936}, 'visibility': 10000, 'wind': {'speed': 2.93, 'deg': 21, 'gust': 2.06}, 'clouds': {'all': 0}, 'dt': 1745225250, 'sys': {'country': 'IN', 'sunrise': 1745196204, 'sunset': 1745241769}, 'timezone': 19800, 'id': 1259229, 'name': 'Pune', 'cod': 200}
> Running step f880e907-ee8f-4240-9c12-d2ef53fef1c1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language t

In [14]:
print(response.response)

The temperature in Pune is 38.41 degrees Celsius.
